In [1]:
import hashlib
import copy

# Clase Nodo

Esta clase simula cada nodo del arbol, tiene un valor, un valor hasheado, un lado que define en que lado esta, un padre, un hermano, y un hijo derecho e izquierdo

In [2]:
class Node:
    def __init__(self, valor):
        
        self.valor = valor
        self.string = valor
        self.lado = None
        self.brother = None
        self.parent = None
        self.hash = None
        self.izquierda = None
        self.derecha = None

# MerkleTree

Esta clase simula el MerkleTree, recibe un funcion de hash y un string

## self.construir

Esta funcion lo que hace es recibir el string y a partir de este crear todas las hojas del arbol, separando el string.

Una vez que tenemos todas las hojas entramos a un while el cual lo que hace es ir concatenando cada par de hermanos hasheado hasta llegar al root, de esta manera obtenemos nuestro MerkleTree.

## self.get_root

Esta funcion retorna el nodo root del arbol, dado mi implementacion de construir es muy facil obtenerlo

## self.get_proof_for

Esta funcion recibe una hoja y retorn una prueba de que la hoja es parte del arbol, dado mi implementacion su funcionamiento es muy simple.

Primero encontramos el objeto de la hoja recibida, una vez encontrado retornamos su hermano, y pasamos a iterar sobre su nodo padre, retornamos su hermano y pasamos a iterar sobre su nodo padre... hasta llegar al nodo root.

In [3]:
class MerkleTree:
    def __init__(self, strings, hash_func):
        self.hash_func = hash_func
        self.leaves = []
        self.root = self.construir(strings)
        

    def get_root(self):
        return self.root

    def get_proof_for(self, item):

        for leave in self.leaves:
            pruebas = [(leave.valor,leave.lado)]
            pruebas_texto = [(leave.valor,leave.lado)]

            if self.hash_func(item.encode('utf-8')).hexdigest() == leave.hash:
                nodo_actual = leave
                while nodo_actual.parent:
                    pruebas.append((nodo_actual.brother.hash,nodo_actual.brother.lado))
                    pruebas_texto.append((nodo_actual.brother.string,nodo_actual.brother.lado))
                    nodo_actual = nodo_actual.parent
                print(pruebas_texto)
                return pruebas
        return None

    def construir(self,strings):
        print_table = []
        nodos = []
        for s in strings:
            nuevo_nodo = Node(s)
            nuevo_nodo.hash = self.hash_func(s.encode('utf-8')).hexdigest()
            nodos.append(nuevo_nodo)
            print_table.append(nuevo_nodo.hash)
            self.leaves.append(nuevo_nodo)

        while len(nodos) > 1:
            cambio = []
            print_table = []
            for index in range(0, len(nodos), 2):
                izquierda = nodos[index]
                if index+1 < len(nodos):
                    derecha = nodos[index+1]
                else:
                    derecha = copy.copy(izquierda)
                derecha.lado = 'd'
                derecha.brother = izquierda
                izquierda.brother = derecha
                izquierda.lado = 'i'
                nuevo_hash = izquierda.hash + derecha.hash
                nuevo_nodo = Node(nuevo_hash)
                nuevo_nodo.string = izquierda.string + derecha.string
                nuevo_nodo.hash = self.hash_func(nuevo_hash.encode('utf-8')).hexdigest()
                nuevo_nodo.izquierda = izquierda
                nuevo_nodo.derecha = derecha
                nuevo_nodo.izquierda.parent = nuevo_nodo
                nuevo_nodo.derecha.parent = nuevo_nodo
                cambio.append(nuevo_nodo)
                print_table.append(nuevo_nodo.hash)
            nodos = cambio
        return nodos[0]


In [4]:
tree = MerkleTree(['1','2','3','4','5','6'],hashlib.sha256)


In [5]:
print(tree.get_proof_for('5'))

[('5', 'i'), ('6', 'd'), ('56', 'd'), ('1234', 'i')]
[('5', 'i'), ('e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683', 'd'), ('43587f59c00a8e528bc7636fabaffcf70cc25afc5b4d53df797faf0dc72f6dd0', 'd'), ('85df8945419d2b5038f7ac83ec1ec6b8267c40fdb3b1e56ff62f6676eb855e70', 'i')]


# Verify

Esta funcion lo que hace es dado una prueba entregada y un item, verifica si este item es aprte del arbol del nodo root.

Su funcionamiento es el siguiente:

Para los primero dos items de la prueba, concatenamos sus hash, segun el orden de derecha e izquierda de los nodos, una vez concatenados, hasheamos su concatenacion, este valor retornado lo concatenamos el el siguiente item del array y lo hasheamos, siempre respetando el orden de derecha e izquierda de los nodos. Todo esto hasta que se termine el array de pruebas, una vez terminado, comparamos el resultado con el nodo root, y retornamos True o False en cada caso.

In [6]:
def verify(root , item , proof):
    
    if proof[0][0] == item:
        if proof[0][1] == 'i':
            actual_texto = proof[0][0] + proof[1][0]
            actual = hashlib.sha256(proof[0][0].encode('utf-8')).hexdigest()  + proof[1][0]
        else:
            actual_texto = proof[1][0] + proof[0][0]
            actual = hashlib.sha256(proof[1][0].encode('utf-8')).hexdigest()  + proof[0][0]
          
        actual = hashlib.sha256(actual.encode('utf-8')).hexdigest()

        

        for node in proof[2::]:
            if node[1] == 'i':
                actual_texto = node[0] + actual_texto
                actual = node[0] + actual
            else:
                actual_texto = actual_texto + node[0]
                actual = actual + node[0]
            
            actual = hashlib.sha256(actual.encode('utf-8')).hexdigest()
        if actual == root:
            return True
        return False
            



In [7]:
print(tree.root.hash)


058bd72c469db066d7b28c9e63e1b7b05c48df9ca23dd521afd0b6154ea47be6


In [8]:
print(verify('058bd72c469db066d7b28c9e63e1b7b05c48df9ca23dd521afd0b6154ea47be6' ,  '5',[('5', 'i'), ('e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683', 'd'), ('43587f59c00a8e528bc7636fabaffcf70cc25afc5b4d53df797faf0dc72f6dd0', 'd'), ('85df8945419d2b5038f7ac83ec1ec6b8267c40fdb3b1e56ff62f6676eb855e70', 'i')]))

True
